In [0]:
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt 
if torch.cuda.is_available():
    avDev = torch.device("cuda")
else:
    avDev = torch.device("cpu")




In [0]:
trainset = dsets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
testset = dsets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

batch_size = 200
n_iters = 2500
num_epochs = n_iters / (len(trainset) / batch_size)
num_epochs = int(num_epochs)

#DataLoading and making it iterable
train_loader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
#Model class
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, 190)
        self.linear2 = nn.Linear(190, output_dim)
        self.relu = nn.Sigmoid()
     
    def forward(self, x):
        out = self.relu(self.linear1(x))
        out = self.linear2(out)
        return out

In [0]:
#Defining input and output dimensions
input_dim = 3*32*32
output_dim = 10
 
model = LogisticRegressionModel(input_dim, output_dim)
model.to(avDev)

In [0]:

#loss class
criterion = nn.CrossEntropyLoss().to(avDev)

best_val=0

#specifying learning rate range to choose the best one
learning_rates = [0.01,0.3]
#finding the best learning rate
for learning_rate in np.linspace(learning_rates[0], learning_rates[1], num = 5):
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    print('Learning rate: {}.'.format(learning_rate))
    iter = 0
    acc_plt=[0]
    confusion_matrix=np.zeros( (10, 10) )
    for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
         
        #training the data
        image =  images.view(-1, 3*32*32).to(avDev)
        label = labels.to(avDev)
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(image)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, label)#
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        acc_plt
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                
                images = images.view(-1, 3*32*32).to(avDev)
              #  kfold = model_selection.KFold(n_splits=5, random_state=seed)
              #  results = model_selection.cross_val_score(model, X, Y, cv=kfold) 
                # Forward pass only to get logits/output
                outputs = model(images)
                 
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                 
                # Total number of labels
                total += labels.size(0)                 
               
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum().float()
                
                #defining confusion matrix
                for i in range(100):
                  confusion_matrix[predicted[i]][labels[i]]+=1
            #Accuracy calculation  
            accuracy = 100. * correct / total
            acc_plt.append(accuracy)
            
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            
            
    val_acc=np.max(acc_plt)
    if val_acc > best_val:
       best_val = val_acc
       best_lr = learning_rate
       best_acc_plt=acc_plt
        
print('Best accurancy: {} and Best learning rate: {}'.format(best_val,best_lr))

In [0]:
print('Confusion Matrix:')
confusion_matrix

In [0]:
#Learning Curve
acc_plt
test_set=np.linspace(0,10000,6)
plt.plot(test_set,acc_plt)
plt.title("Learning Curve")
plt.xlabel("Test Set Size"), plt.ylabel("Accuracy Score")
plt.tight_layout()
plt.show()